In [19]:
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import optuna
from optuna.pruners import MedianPruner
from itertools import cycle
import copy
import mlflow
import math

In [20]:
mlflow.set_tracking_uri("http://localhost:8080")

In [22]:
def get_or_create_experiment(experiment_name):
  """
  Retrieve the ID of an existing MLflow experiment or create a new one if it doesn't exist.

  This function checks if an experiment with the given name exists within MLflow.
  If it does, the function returns its ID. If not, it creates a new experiment
  with the provided name and returns its ID.

  Parameters:
  - experiment_name (str): Name of the MLflow experiment.

  Returns:
  - str: ID of the existing or newly created MLflow experiment.
  """

  if experiment := mlflow.get_experiment_by_name(experiment_name):
      return experiment.experiment_id
  else:
      return mlflow.create_experiment(experiment_name)

In [24]:
experiment_id = get_or_create_experiment("mean teacher")

# Set the current active MLflow experiment
mlflow.set_experiment(experiment_id=experiment_id)

# override Optuna's default logging to ERROR only
optuna.logging.set_verbosity(optuna.logging.ERROR)

MlflowException: API request to http://localhost:8080/api/2.0/mlflow/experiments/get-by-name failed with exception HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /api/2.0/mlflow/experiments/get-by-name?experiment_name=mean+teacher (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000286FE156BE0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [ ]:
import mlflow
from mlflow.optuna.storage import MlflowStorage

experiment_id = mlflow.get_experiment_by_name('test').experiment_id

mlflow_storage = MlflowStorage(experiment_id=experiment_id)

In [ ]:
def champion_callback(study, frozen_trial):
  """
  Logging callback that will report when a new trial iteration improves upon existing
  best trial values.

  Note: This callback is not intended for use in distributed computing systems such as Spark
  or Ray due to the micro-batch iterative implementation for distributing trials to a cluster's
  workers or agents.
  The race conditions with file system state management for distributed trials will render
  inconsistent values with this callback.
  """

  winner = study.user_attrs.get("winner", None)

  if study.best_value and winner != study.best_value:
      study.set_user_attr("winner", study.best_value)
      if winner:
          improvement_percent = (abs(winner - study.best_value) / study.best_value) * 100
          print(
              f"Trial {frozen_trial.number} achieved value: {frozen_trial.value} with "
              f"{improvement_percent: .4f}% improvement"
          )
      else:
          print(f"Initial trial {frozen_trial.number} achieved value: {frozen_trial.value}")

In [11]:
def add_gaussian_noise(data_tensor, std=0.1):
    """Adds Gaussian noise to a PyTorch tensor."""
    noise = torch.randn_like(data_tensor) * std
    return data_tensor + noise

In [12]:
class SimpleNN(nn.Module):
    def __init__(self, input_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        return x

In [13]:
iris = load_iris()
# sufle dataset before split
np.random.seed(42)
indices = np.random.permutation(len(iris.data))
iris.data = iris.data[indices]
iris.target = iris.target[indices]
X = iris.data
y = iris.target

X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

full_dataset = TensorDataset(X_tensor, y_tensor)

# Split data into training and testing sets
train_data, test_data = train_test_split(full_dataset, test_size=0.2, random_state=42)

train_data, validation_data = train_test_split(train_data, test_size=0.2, random_state=42)
print(X_tensor.shape, y_tensor.shape)


batch_size = 1024

val_loader = DataLoader(validation_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

torch.Size([150, 4]) torch.Size([150])


In [ ]:
# start an optuna study to minimize the validation loss
def objective(trial):
    with mlflow.start_run(nested=True):
    
        params = {
            'batch_size_train': trial.suggest_categorical('batch_size_train', [16, 32, 64, 128, 256, 512, 1024])
            ,'lr': trial.suggest_float('lr', 1e-5, 1e-1, log=True)
            ,'num_epochs': trial.suggest_int('num_epochs', 50, 500)
        }
        
        batch_size_train = params['batch_size_train']
        train_loader = DataLoader(train_data, batch_size=batch_size_train, shuffle=True)
        input_size = 4
        output_size = 3
        model = SimpleNN(input_size, output_size)
        loss_function = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=params['lr'])

        for epoch in range(params['num_epochs']):
            # Training
            model.train()
            correct_train = 0
            total_train = 0
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                _, predicted_train = torch.max(outputs.data, 1)
                total_train += labels.size(0)
                correct_train += (predicted_train == labels).sum().item()
                loss = loss_function(outputs, labels)
                loss.backward()
                optimizer.step()
            validation_accuracy = correct_val / total_val

            # Evaluate on validation set
            model.eval()
            correct_val = 0
            total_val = 0
            with torch.no_grad():
                for val_inputs, val_labels in val_loader:
                    val_outputs = model(val_inputs)
                    _, predicted_val = torch.max(val_outputs.data, 1)
                    total_val += val_labels.size(0)
                    correct_val += (predicted_val == val_labels).sum().item()
            validation_accuracy = correct_val / total_val
            trial.report(validation_accuracy, epoch)
            
            if trial.should_prune():
                raise optuna.TrialPruned()
        
        mlflow.log_params(params)
        mlflow.log_metric('train accuracy', validation_accuracy)
        mlflow.log_metric('validation accuracy', validation_accuracy)
        
            
    return validation_accuracy


In [16]:
# 1. Define the storage location and a name for your study
storage_name = "sqlite:///my_optuna_study.db"
study_name = "supervised_iris_study"
pruner = MedianPruner(n_startup_trials=5, n_warmup_steps=30, interval_steps=10)
# Create a study object and specify the direction is to maximize accuracy.
study = optuna.create_study(direction='maximize', pruner=pruner, storage=storage_name, study_name=study_name, load_if_exists=True)

[I 2025-09-29 14:04:39,601] A new study created in RDB with name: supervised_iris_study


In [ ]:
# Start the optimization. Optuna will run the objective function 100 times.
study.optimize(objective, n_trials=1000, callbacks=[champion_callback])

mlflow.log_params(study.best_params)
mlflow.log_metric("best_mse", study.best_value)
mlflow.log_metric("best_rmse", math.sqrt(study.best_value))

# Log tags
mlflow.set_tags(
    tags={
        "project": "Mean Teacher",
        "optimizer_engine": "optuna",
        "model_family": "pytorch",
        "feature_set_version": 1,
    }
)

artifact_path = "model"
model = 
mlflow.pytorch.log_model(pytorch_model=model, name=artifact_path, input_example=train_data[0], model_format='ubj', metadata={'model_data_version': 1},)

model_uri = mlflow.get_artifac_uri(artifact_path)

loaded = mlflow.xgboost.load_model(model_uri)

[I 2025-09-29 14:04:56,359] Trial 0 finished with value: 0.4583333333333333 and parameters: {'batch_size_train': 32, 'lr': 0.0005377518515919115, 'num_epochs': 395}. Best is trial 0 with value: 0.4583333333333333.
[I 2025-09-29 14:05:02,338] Trial 1 finished with value: 0.8333333333333334 and parameters: {'batch_size_train': 16, 'lr': 0.00626446907455853, 'num_epochs': 160}. Best is trial 1 with value: 0.8333333333333334.
[I 2025-09-29 14:05:08,873] Trial 2 finished with value: 0.5416666666666666 and parameters: {'batch_size_train': 64, 'lr': 4.941433846982852e-05, 'num_epochs': 272}. Best is trial 1 with value: 0.8333333333333334.
[I 2025-09-29 14:05:20,048] Trial 3 finished with value: 0.5833333333333334 and parameters: {'batch_size_train': 64, 'lr': 0.0012520065381267458, 'num_epochs': 464}. Best is trial 1 with value: 0.8333333333333334.
[I 2025-09-29 14:05:26,357] Trial 4 finished with value: 0.16666666666666666 and parameters: {'batch_size_train': 16, 'lr': 4.938551275989298e-05,

KeyboardInterrupt: 

In [ ]:
# Print the best hyperparameters found
best_params = study.best_params
print(f"Best hyperparameters: {best_params}")

# Print the best score achieved
best_value = study.best_value
print(f"Best cross-validation accuracy: {best_value:.4f}")

# You can also get the best trial object itself for more details
best_trial = study.best_trial
print(f"Best trial number: {best_trial.number}")

ValueError: No trials are completed yet.

In [ ]:
# Visualize the optimization history
optuna.visualization.plot_optimization_history(study).show()

# Visualize the importance of each hyperparameter
optuna.visualization.plot_param_importances(study).show()

Epoch [10/1500], Loss: 1.1172
Validation Loss: 1.0355
Best model saved!
Epoch [20/1500], Loss: 1.0169
Validation Loss: 1.0044
Best model saved!
Epoch [30/1500], Loss: 0.8882
Validation Loss: 0.9597
Best model saved!
Epoch [40/1500], Loss: 0.7929
Validation Loss: 0.9178
Best model saved!
Epoch [50/1500], Loss: 0.8094
Validation Loss: 0.8965
Best model saved!
Epoch [60/1500], Loss: 0.7888
Validation Loss: 0.8818
Best model saved!
Epoch [70/1500], Loss: 0.6870
Validation Loss: 0.8675
Best model saved!
Epoch [80/1500], Loss: 0.6844
Validation Loss: 0.8736
Epoch [90/1500], Loss: 0.6467
Validation Loss: 0.8644
Best model saved!
Epoch [100/1500], Loss: 0.6725
Validation Loss: 0.8553
Best model saved!
Epoch [110/1500], Loss: 0.6672
Validation Loss: 0.8475
Best model saved!
Epoch [120/1500], Loss: 0.6257
Validation Loss: 0.8262
Best model saved!
Epoch [130/1500], Loss: 0.6366
Validation Loss: 0.8159
Best model saved!
Epoch [140/1500], Loss: 0.6313
Validation Loss: 0.8030
Best model saved!
Epoch

In [30]:
# Assuming you have loaded the best_model.pth after training is complete
best_model_path = 'best_model.pth'
model.load_state_dict(torch.load(best_model_path))
model.eval()

correct = 0
total = 0
with torch.no_grad():
    for test_inputs, test_labels in test_loader:
        test_outputs = model(test_inputs)
        _, predicted = torch.max(test_outputs.data, 1)
        total += test_labels.size(0)
        correct += (predicted == test_labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the model on the test data: {accuracy:.2f}%')

Accuracy of the model on the test data: 100.00%


test now with half of the dataset of size

In [31]:
iris = load_iris()
# shuffle dataset before split
np.random.seed(42)
indices = np.random.permutation(len(iris.data))
iris.data = iris.data[indices]
iris.target = iris.target[indices]
X = iris.data
y = iris.target

X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

full_dataset = TensorDataset(X_tensor, y_tensor)

# split with label and unlabel data. Assume 20% labeled data
label_data, unlabel_data = train_test_split(full_dataset, test_size=0.8, random_state=42)

# Split data into training and testing sets
train_data, test_data = train_test_split(label_data, test_size=0.2, random_state=42)

train_data, validation_data = train_test_split(train_data, test_size=0.2, random_state=42)
print(X_tensor.shape, y_tensor.shape)

batch_size_train = 64
batch_size = 1024

train_loader = DataLoader(train_data, batch_size=batch_size_train, shuffle=True)
val_loader = DataLoader(validation_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
unlabeled_loader = DataLoader(unlabel_data, batch_size=batch_size, shuffle=True)

torch.Size([150, 4]) torch.Size([150])


In [32]:
input_size = 4
output_size = 3
model = SimpleNN(input_size, output_size)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [44]:
num_epochs = 1500
best_val_loss = float('inf')
for epoch in range(num_epochs):
    # Training
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
        # Evaluate on validation set
        model.eval()
        with torch.no_grad():
            val_losses = []
            for val_inputs, val_labels in val_loader:
                val_outputs = model(val_inputs)
                val_loss = loss_function(val_outputs, val_labels)
                val_losses.append(val_loss.item())
            avg_val_loss = np.mean(val_losses)
            print(f'Validation Loss: {avg_val_loss:.4f}')
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                torch.save(model.state_dict(), 'best_model_label.pth')
                print('Best model saved!')


Epoch [10/1500], Loss: 0.4211
Validation Loss: 0.5308
Best model saved!
Epoch [20/1500], Loss: 0.4211
Validation Loss: 0.5308
Epoch [30/1500], Loss: 0.4211
Validation Loss: 0.5308
Epoch [40/1500], Loss: 0.4211
Validation Loss: 0.5308
Epoch [50/1500], Loss: 0.4211
Validation Loss: 0.5308
Epoch [60/1500], Loss: 0.4211
Validation Loss: 0.5308
Epoch [70/1500], Loss: 0.4211
Validation Loss: 0.5308
Epoch [80/1500], Loss: 0.4211
Validation Loss: 0.5308
Epoch [90/1500], Loss: 0.4211
Validation Loss: 0.5308
Epoch [100/1500], Loss: 0.4211
Validation Loss: 0.5308
Epoch [110/1500], Loss: 0.4211
Validation Loss: 0.5308
Epoch [120/1500], Loss: 0.4211
Validation Loss: 0.5308
Epoch [130/1500], Loss: 0.4211
Validation Loss: 0.5308
Epoch [140/1500], Loss: 0.4211
Validation Loss: 0.5308
Epoch [150/1500], Loss: 0.4211
Validation Loss: 0.5308
Epoch [160/1500], Loss: 0.4211
Validation Loss: 0.5308
Epoch [170/1500], Loss: 0.4211
Validation Loss: 0.5308
Epoch [180/1500], Loss: 0.4211
Validation Loss: 0.5308
E

In [34]:
# Assuming you have loaded the best_model.pth after training is complete
best_model_path = 'best_model_label.pth'
model.load_state_dict(torch.load(best_model_path))
model.eval()

correct = 0
total = 0
with torch.no_grad():
    for test_inputs, test_labels in test_loader:
        test_outputs = model(test_inputs)
        _, predicted = torch.max(test_outputs.data, 1)
        total += test_labels.size(0)
        correct += (predicted == test_labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the model on the test data: {accuracy:.2f}%')

Accuracy of the model on the test data: 66.67%


In [63]:
# Assuming you have loaded the best_model.pth after training is complete
best_model_path = 'best_model_label.pth'
model.load_state_dict(torch.load(best_model_path))
model.eval()

correct = 0
total = 0
with torch.no_grad():
    for unlabel_inputs, unlabel_labels in unlabeled_loader:
        unlabel_outputs = model(unlabel_inputs)
        _, predicted = torch.max(unlabel_outputs.data, 1)
        total += unlabel_labels.size(0)
        correct += (predicted == unlabel_labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the model on the unlabel data: {accuracy:.2f}%')

Accuracy of the model on the unlabel data: 79.17%


try using mean teacher

In [37]:
import copy

In [58]:
def update_teacher_weights(student_model, teacher_model, alpha=0.99):
    for teacher_param, student_param in zip(teacher_model.parameters(), student_model.parameters()):
        teacher_param.data.mul_(alpha).add_(student_param.data, alpha=1 - alpha)

In [64]:
input_size = 4
output_size = 3
student_model = SimpleNN(input_size, output_size)
# Instantiate the teacher model with the same architecture
teacher_model = copy.deepcopy(student_model)

# The teacher model should not have its gradients calculated
for param in teacher_model.parameters():
    param.requires_grad = False
supervised_loss_fn = nn.CrossEntropyLoss()
consistency_loss_fn = nn.MSELoss()
optimizer = optim.Adam(student_model.parameters(), lr=0.001)
alpha = 0.99  # for exponential moving average
lambda_u = 0.3  # weight for unlabeled loss

In [66]:
from itertools import cycle

In [ ]:
# Training loop
num_epochs = 1000
best_val_loss = float('inf')
for epoch in range(num_epochs):
    student_model.train()
    for (labeled_data, labeled_labels), (unlabeled_data,_) in zip(cycle(train_loader), unlabeled_loader):
        # Supervised loss
        labeled_outputs = student_model(labeled_data)
        supervised_loss = supervised_loss_fn(labeled_outputs, labeled_labels)
        
        # 1. Weakly augmented view for the TEACHER
        unlabeled_data_teacher = add_gaussian_noise(unlabeled_data, std=0.05)
        
        # 2. Strongly augmented view for the STUDENT
        unlabeled_data_student = add_gaussian_noise(unlabeled_data, std=0.15)
        
        # Consistency loss
        with torch.no_grad():
            teacher_outputs = teacher_model(unlabeled_data_teacher)
        student_outputs_unlabeled = student_model(unlabeled_data_student)
        consistency_loss = consistency_loss_fn(student_outputs_unlabeled, teacher_outputs)

        total_loss = supervised_loss + lambda_u * consistency_loss
        
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        # Update teacher model's weights using EMA
        update_teacher_weights(student_model, teacher_model)
        
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss .item():.4f}')
            # Evaluate on validation set
            student_model.eval()
            with torch.no_grad():
                val_losses = []
                for val_inputs, val_labels in val_loader:
                    val_outputs = student_model(val_inputs)
                    val_loss = loss_function(val_outputs, val_labels)
                    val_losses.append(val_loss.item())
                avg_val_loss = np.mean(val_losses)
                print(f'Validation Loss: {avg_val_loss:.4f}')
                if avg_val_loss < best_val_loss:
                    best_val_loss = avg_val_loss
                    torch.save(student_model.state_dict(), 'best_model_label_and_unlabel.pth')
                    print('Best model saved!')


Epoch [10/1000], Loss: 1.2190
Validation Loss: 1.5891
Best model saved!
Epoch [20/1000], Loss: 1.1657
Validation Loss: 1.5130
Best model saved!
Epoch [30/1000], Loss: 1.1193
Validation Loss: 1.4467
Best model saved!
Epoch [40/1000], Loss: 1.0793
Validation Loss: 1.3888
Best model saved!
Epoch [50/1000], Loss: 1.0418
Validation Loss: 1.3376
Best model saved!
Epoch [60/1000], Loss: 1.0052
Validation Loss: 1.2921
Best model saved!
Epoch [70/1000], Loss: 0.9700
Validation Loss: 1.2502
Best model saved!
Epoch [80/1000], Loss: 0.9366
Validation Loss: 1.2105
Best model saved!
Epoch [90/1000], Loss: 0.9044
Validation Loss: 1.1729
Best model saved!
Epoch [100/1000], Loss: 0.8738
Validation Loss: 1.1370
Best model saved!
Epoch [110/1000], Loss: 0.8433
Validation Loss: 1.1026
Best model saved!
Epoch [120/1000], Loss: 0.8161
Validation Loss: 1.0700
Best model saved!
Epoch [130/1000], Loss: 0.7900
Validation Loss: 1.0390
Best model saved!
Epoch [140/1000], Loss: 0.7647
Validation Loss: 1.0096
Best 

In [68]:
# Assuming you have loaded the best_model.pth after training is complete
best_model_path = 'best_model_label_and_unlabel.pth'
model.load_state_dict(torch.load(best_model_path))
model.eval()

correct = 0
total = 0
with torch.no_grad():
    for test_inputs, test_labels in test_loader:
        test_outputs = model(test_inputs)
        _, predicted = torch.max(test_outputs.data, 1)
        total += test_labels.size(0)
        correct += (predicted == test_labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the model on the test data: {accuracy:.2f}%')

Accuracy of the model on the test data: 100.00%


In [69]:
# Assuming you have loaded the best_model.pth after training is complete
best_model_path = 'best_model_label_and_unlabel.pth'
model.load_state_dict(torch.load(best_model_path))
model.eval()

correct = 0
total = 0
with torch.no_grad():
    for unlabel_inputs, unlabel_labels in unlabeled_loader:
        unlabel_outputs = model(unlabel_inputs)
        _, predicted = torch.max(unlabel_outputs.data, 1)
        total += unlabel_labels.size(0)
        correct += (predicted == unlabel_labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the model on the unlabel data: {accuracy:.2f}%')

Accuracy of the model on the unlabel data: 95.00%
